## Test with straight beam

The aim of this test is to compare the inertial loads of an analytical case with the ones given by the code.

The beam used is straight, measures 1 meter, weighs 100 kg and it only has two nodes. It is defined with the following half chord position and property files.


The stiffness matrix is not needed in this test, but, to make the structural code work, a valid matrix was used from the first node of the original IEA 15 MW reference WT file.

In [53]:
"""
Using a straight beam.
"""

# Import libraries
import os
import sys
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from beam_corot.ComplBeam import ComplBeam
from beam_corot.CoRot import CoRot
from utils import save_load
from inertial_forces_v2 import inertial_loads_fun


In [60]:

# Model input json file name
f_model_json = "straight_beam_no_offset_static.json"

# Input files folder
inputfolder = os.path.join(os.getcwd(),'straight_beam')
mainfile = os.path.join(inputfolder,f_model_json)

# Extra geometry info
hub_di = 0 # Hub diameter [m]
# 80 m of distance of the beam from the axis of rotation

In [55]:
# - Instantiate beam
save_load([0], inputfolder, onlyy=True) # Creates a force file
beam_no_offset = ComplBeam(mainfile)

# - Get the radius location of nodes
r = beam_no_offset.nodeLocations[:,2] # z-axis position
beam_no_offset.nodeLocations

-----------
ComplBeam Model Created
Static analysis done


array([[0., 0., 0.],
       [0., 0., 1.]])

In [56]:
# - Get the mass matrix
# For non dynamic calculations the mass matrix is not calculated
f_model_json = "straight_beam_no_offset_dynamic.json"
mainfile_dyn = os.path.join(inputfolder,f_model_json)

# Calculate mass matrix
beam_no_offset = ComplBeam(mainfile_dyn)
M_mat_no_offset = beam_no_offset.M_mat_full

# Display DataFrame in Jupyter notebook
M_mat_no_offset_df = pd.DataFrame(M_mat_no_offset)
M_mat_no_offset_df

-----------
**********                  No "mass_matrix" type defined                    **********
**********          "mass_matrix" = "Timo" / "Compl" (default = "Timo")        **********
Timo Mass Matrix
ComplBeam Model Created
Dynamic analysis done


,0,1,2,3,4,5,6,7,8,9,10,11
0,33.500180,-0.207198,0.000000,0.042634,4.209301,-0.093673,16.499820,0.207198,0.000000,0.042634,-4.124033,0.093673
1,-0.207198,33.580883,0.000000,-4.229476,-0.062810,-0.137028,0.207198,16.419117,0.000000,4.103857,-0.062810,0.137028
2,0.000000,0.000000,33.333333,0.000000,0.000000,0.000000,0.000000,0.000000,16.666667,0.000000,0.000000,0.000000
3,0.042634,-4.229476,0.000000,0.834256,0.000922,0.000751,-0.042634,-4.103857,0.000000,-0.832411,0.000922,-0.000751
4,4.209301,-0.062810,0.000000,0.000922,0.834256,0.000751,4.124033,0.062810,0.000000,0.000922,-0.832411,-0.000751
5,-0.093673,-0.137028,0.000000,0.000751,0.000751,0.004702,0.093673,0.137028,0.000000,0.000751,0.000751,-0.004702
6,16.499820,0.207198,0.000000,-0.042634,4.124033,0.093673,33.500180,-0.207198,0.000000,-0.042634,-4.209301,-0.093673
7,0.207198,16.419117,0.000000,-4.103857,0.062810,0.137028,-0.207198,33.580883,0.000000,4.229476,0.062810,-0.137028
8,0.000000,0.000000,16.666667,0.000000,0.000000,0.000000,0.000000,0.000000,33.333333,0.000000,0.000000,0.000000
9,0.042634,4.103857,0.000000,-0.832411,0.000922,0.000751,-0.042634,4.229476,0.000000,0.834256,0.000922,-0.000751


## Calculating centrifugal force

Let's calculate the acceleration in the node position which is also the CoG.

The operational condicions are:

    RPM = 7
    pitch = 0


In [62]:
# Operations info
omega = 1 # rad/s
pitch_deg = 0
pitch_rad = np.deg2rad(pitch_deg)


First, the posotions are needed. This is set manually because it is very simple.

In [58]:
pos = np.array([[0,0,0,0,0,0],[0,0,1,0,0,0]])
pos.flatten()

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0])

In [63]:
# Calculate inertial loads
load = -inertial_loads_fun(pos,M_mat_no_offset,hub_di,omega,pitch_rad)
load = np.reshape(load, (-1,6))
load

array([[-0.        , -0.        , 16.66666667, -0.        , -0.        ,
        -0.        ],
       [-0.        , -0.        , 33.33333333, -0.        , -0.        ,
        -0.        ]])

## Analytical result

This problem is really simple on paper. There is a beam measuring 1 meter, weighing a 100 kg and rotating at a constant 1 rad/s around one of its ends.

The centrifugar force in its center of mass is $F=m\cdot \omega^{2}\cdot r= 100 \cdot 1 \cdot 0.5 = 50 \text{ N}$. This force is outwards, meaning positive z.

This is equal to the summe of the force of both nodes!

## Adding an offset in the centre of gravity

In [64]:
# - Get the mass matrix
# For non dynamic calculations the mass matrix is not calculated
f_model_json = "straight_beam_offset_dynamic.json"
mainfile_dyn = os.path.join(inputfolder,f_model_json)

# Calculate mass matrix
beam_offset = ComplBeam(mainfile_dyn)
M_mat_offset = beam_offset.M_mat_full

# Display DataFrame in Jupyter notebook
M_mat_offset_df = pd.DataFrame(M_mat_offset)
M_mat_offset_df

-----------
**********                  No "mass_matrix" type defined                    **********
**********          "mass_matrix" = "Timo" / "Compl" (default = "Timo")        **********
Timo Mass Matrix
ComplBeam Model Created
Dynamic analysis done


,0,1,2,3,4,5,6,7,8,9,10,11
0,33.500180,-0.207198,-0.097894,0.042634,4.209301,-0.093673,16.499820,0.207198,-0.146841,0.042634,-4.124033,0.081480
1,-0.207198,33.580883,0.097894,-4.229476,-0.062810,-0.970361,0.207198,16.419117,0.146841,4.103857,-0.062810,-0.684112
2,-0.097894,0.097894,33.333333,-0.048947,0.784386,0.114210,0.097894,-0.097894,16.666667,-0.048947,0.784386,-0.114210
3,0.042634,-4.229476,-0.048947,0.834256,0.000922,0.167418,-0.042634,-4.103857,-0.073421,-0.832411,0.000922,0.243153
4,4.209301,-0.062810,0.784386,0.000922,0.834256,0.000751,4.124033,0.062810,0.759913,0.000922,-0.832411,-0.006847
5,-0.093673,-0.970361,0.114210,0.167418,0.000751,0.004702,0.093673,-0.696305,0.171315,-0.165916,0.000751,-0.018254
6,16.499820,0.207198,0.097894,-0.042634,4.124033,0.093673,33.500180,-0.207198,0.146841,-0.042634,-4.209301,-0.081480
7,0.207198,16.419117,-0.097894,-4.103857,0.062810,-0.696305,-0.207198,33.580883,-0.146841,4.229476,0.062810,-2.649221
8,-0.146841,0.146841,16.666667,-0.073421,0.759913,0.171315,0.146841,-0.146841,33.333333,-0.073421,2.426579,-0.171315
9,0.042634,4.103857,-0.048947,-0.832411,0.000922,-0.165916,-0.042634,4.229476,-0.073421,0.834256,0.000922,-0.256847


In [66]:
pos_cg = np.array([[0,0,0,0,0,0],[0.1,0,1,0,0,0]])
# Calculate inertial loads
load = -inertial_loads_fun(pos_cg,M_mat_offset,hub_di,omega,pitch_rad)
load = np.reshape(load, (-1,6))
load

array([[ 1.50314079,  0.16756101, 16.67645608, -0.07768401,  1.17231599,
         0.18068208],
       [ 3.49685921, -0.16756101, 33.34801746, -0.07768401,  2.00564932,
        -0.17946278]])

## Question

I'm stuck with the analytical answer to this question. How can I compare this case, where the fucntion